<a href="https://colab.research.google.com/github/ftay16/Data-Science-Portfolio/blob/main/SQL_Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# bring in relevant libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# read data into dataframe from excel
movie = pd.read_excel('dataset.xlsx', sheet_name = 'movie')
movie

,mID,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,NaN
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [ ]:
reviewer = pd.read_excel('dataset.xlsx', sheet_name = 'reviewer')
reviewer

,rID,name
0,201,Sarah Martinez
1,202,Daniel Lewis
2,203,Brittany Harris
3,204,Mike Anderson
4,205,Chris Jackson
5,206,Elizabeth Thomas
6,207,James Cameron
7,208,Ashley White


In [ ]:
rating = pd.read_excel('dataset.xlsx', sheet_name = 'rating')
rating

,rID,mID,stars,ratingDate
0,201,101,2,2012-01-22
1,201,101,4,2013-01-27
2,202,106,4,NaT
3,203,103,2,2008-01-20
4,203,108,4,2002-01-12
5,203,108,2,2009-01-30
6,204,101,3,2010-01-09
7,205,103,3,2010-01-27
8,205,104,2,2010-01-22
9,205,108,4,NaT


In [ ]:
# attach the dataframe to sql
engine = create_engine('sqlite://', echo = False)

movie.to_sql('movie', con = engine)
reviewer.to_sql('reviewer', con = engine)
rating.to_sql('rating', con = engine)


14

In [ ]:
# 1. Find the titles of all movies directed by Steven Spielberg.
sql_1 = """
select
distinct title
from movie
where director = 'Steven Spielberg'
"""

pd.read_sql_query(sql_1, con = engine)

,title
0,E.T.
1,Raiders of the Lost Ark


In [ ]:
# 2. Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.

sql_2 = """
select
distinct year
from (
    select
    year, stars
    from rating r
    left join movie m
    on r.mID = m.mID
    group by 1, 2
    having stars > 3
)
order by 1 asc
"""
pd.read_sql_query(sql_2, con = engine)

,year
0,1937
1,1939
2,1981
3,2009


In [ ]:
# 3. Find the titles of all movies that have no ratings

sql_3 = """
select
distinct title
from (
    select
    title, stars
    from movie m
    left join rating r
    on m.mID = r.mID
    where stars is null
    group by 1, 2
)
"""

pd.read_sql_query(sql_3, con = engine)

,title
0,Star Wars
1,Titanic


In [ ]:
# 4. For each movie that has at least one rating, find the highest number of stars that movie received.
# Return the movie title and number of stars. Sort by movie title.

sql_4 = """
select
title, max(stars) as highest_stars
from rating r
left join movie m
on m.mID = r.mID
group by 1
"""

pd.read_sql_query(sql_4, con = engine)

,title,highest_stars
0,Avatar,5
1,E.T.,3
2,Gone with the Wind,4
3,Raiders of the Lost Ark,4
4,Snow White,5
5,The Sound of Music,3


In [ ]:
# 5. For each movie that has at least one rating,
# find the movie title and total number of stars, the highest star and the person who gave highest star.

sql_5 = """
with rv_ as (
    select
    rt.rID, rv.name, rt.mID, max(rt.stars) as highest_stars
    from rating rt
    left join reviewer rv
    on rt.rID = rv.rID
    group by 1, 2, 3

)
, rt_ as (
    select m.title, rt.mID, max(rt.stars) as highest_stars, sum(rt.stars) as total_stars
    from rating rt
    left join movie m
    on rt.mID = m.mID
    group by 1, 2
)

select
title, name, rt_.highest_stars, rt_.total_stars
from rt_
inner join rv_
on rt_.mID = rv_.mID and rt_.highest_stars = rv_.highest_stars
"""

pd.read_sql_query(sql_5, con = engine)

,title,name,highest_stars,total_stars
0,Avatar,James Cameron,5,8
1,E.T.,Ashley White,3,5
2,Gone with the Wind,Sarah Martinez,4,9
3,Raiders of the Lost Ark,Brittany Harris,4,10
4,Raiders of the Lost Ark,Chris Jackson,4,10
5,Snow White,Elizabeth Thomas,5,9
6,The Sound of Music,Chris Jackson,3,5


In [ ]:
# 6. For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time,
# return the reviewer's name and the title of the movie.

sql_6 = """
with freq as (
    select title, name, stars, ratingDate, row_number() over (partition by name, title order by rt.ratingDate asc) as r
    from
    rating rt
    left join reviewer rv on rt.rID = rv.rID
    left join movie m on rt.mID = m.mID
)

, freq_1 as (
    select title, name from freq where r = 2
)


select
freq.name
, freq.title
, sum(case when r = 1 then stars else 0 end) first_rating
, sum(case when r = 2 then stars else 0 end) second_rating
from freq
inner join freq_1
on freq.title = freq_1.title
and freq.name = freq_1.name
group by 1, 2
having first_rating < second_rating
"""

pd.read_sql_query(sql_6, con = engine)

,name,title,first_rating,second_rating
0,Sarah Martinez,Gone with the Wind,2,4


In [ ]:
# 7. For each movie, return the title and the 'rating spread', that is, the difference between highest and lowest ratings given to that movie.
# Sort by rating spread from highest to lowest, then by movie title.

sql_7 = """
select title, max(stars) - min(stars) as rating_spread
from rating rt
left join
movie m
on rt.mID = m.mID
group by 1
order by 2 DESC, 1 ASC

"""

pd.read_sql_query(sql_7, con = engine)

,title,rating_spread
0,Avatar,2
1,Gone with the Wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
5,The Sound of Music,1


In [ ]:
# 8. Find the names of reviewers for every director (one row per director with all reviewers)

sql_8 = """
select
--distinct
m.director, group_concat(rv.name) as name
from
rating rt
left join movie m on rt.mID = m.mID
left join reviewer rv on rt.rID = rv.rID
group by 1
order by 1
"""

pd.read_sql_query(sql_8, con = engine)

,director,name
0,None,"Daniel Lewis,Elizabeth Thomas"
1,James Cameron,"Elizabeth Thomas,James Cameron"
2,Robert Wise,"Brittany Harris,Chris Jackson"
3,Steven Spielberg,"Brittany Harris,Brittany Harris,Chris Jackson,..."
4,Victor Fleming,"Sarah Martinez,Sarah Martinez,Mike Anderson"
